Todo: sloučit povídky povídkových filmů do 1 řádku?

In [1]:
import re
import os
import json
import pandas as pd

In [2]:
with open(os.path.join("data_fixes", "opravy.json"), encoding="utf-8") as opravy:
    opravy = json.loads(opravy.read())

In [3]:
with open(os.path.join("data_fixes", "seznamy.json"), encoding="utf-8") as seznamy:
    seznamy = json.loads(seznamy.read())
profese = seznamy["profese"]

In [4]:
df = pd.read_json(os.path.join("data", "filmy_raw.json"))

In [5]:
df[df["Film"] == "Temné slunce (397277)"]["Kaskadéři"].to_list()

['126893, 1502, 14390, 27964, 8249, 88491, 124371, 22041, 7952, 126451, 76594, 67247, 126585, 2609, 126622, 2615, 11810, 7960, 11806, 11812, 105121, Martina Bártová, Stanislav Benda, Irena Farkašová, Iva Fousková, Zdeněk Havlík, Josef Hejzlar, Lucie Horáková, Světlana Hovorková, Dalibor Jindříšek, Ludmila Kellerová, Petr Klír, Jana Koliášová, Miroslav Kotek, Zbyněk Krákora, Pavel Kunáček, Lenka Lajčíková, František Linhart, Zdeněk Linhart, Kateřina Milerová, Eva Pavelková, Jana Pipková, Zdeněk Plavec, Helena Průšová, Stanislav Sedlařík, Jiří Sládek, Věra Starečková, Johana Stuchlíková, Zdeněk Šiška, Iva Špačková, Marcela Šlenhofová, Jindřiška Špičková, Jiří Vach, Pavel Valenta, Zdena Valterová, Blanka Vykydalová, Pavel Želízko']

In [6]:
df

,Film,Země původu,Copyright,Rok výroby,Premiéra,Minutáž,Režie,Kategorie,Žánr,Typologie,...,Výtvarník pozadí,Výrobní skupina,Zákazník,Vývozní společnost,Umělecké vedení,Zpěv role,Překlad české verze,Rychlost,Přístupnost,cyklus
0,Tábor padlých žen (10144),"Slovensko, ...",1997,1997,premiéra 28. 9. 2000 /doporučená přístupnost o...,100 min,", Laco Halama",film,"historický, tragikomedie","hraný, distribuční",...,None,None,None,None,None,None,None,None,None,None
1,Eastern – Krvavý Hugo (10173),Česká republika,1997,1997,nedistribuční premiéra 27. 4. 1997 ...,6 min,"1189,",film,groteska,"animovaný, distribuční, studentský",...,None,None,None,None,None,None,None,None,None,None
2,O Ječmínkovi (102282),Česká republika,2003,2002,nedistribuční premiéra 18. 4. 2003 (televizní...,61 min,"3593,",film,pohádka,"hraný, distribuční",...,None,None,None,None,None,None,None,None,None,None
3,Zimní víla (102314),Česká republika,1999,1999,nedistribuční premiéra 24. 12. 1999 (televizn...,52 min,"1055,",film,pohádka,"hraný, distribuční",...,None,None,None,None,None,None,None,None,None,None
4,Narkoblues (10242),"Česká republika, ...",1997,1997,premiéra 10. 11. 2016 /přístupné bez omezení/,9 min,"3113, 3030,",film,osvětový,"animovaný, distribuční",...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4533,Hotel Sarajevo aneb Sny obyčejného svatebčana ...,Česká republika,1997,1997,premiéra 12. 8. 1997,None,"3466,",film,povídka,"hraný, distribuční",...,None,None,None,None,None,None,None,None,None,None
4534,Zdivočelá země (9738),Česká republika,1997,1997,premiéra 13. 11. 1997 /přístupný/ (kino 64 U H...,114 min,"2947,",film,"dobrodružný, historický","hraný, distribuční",...,None,None,None,None,None,None,None,None,None,None
4535,Biograf (9769),Česká republika,1997,1997,nedistribuční premiéra 12. 6. 1997 ...,5 min,"7957,",film,anekdota,"animovaný, distribuční, studentský",...,None,None,None,None,None,None,None,None,None,None
4536,Knoflíkáři (9793),Česká republika,1997,1997,premiéra 20. 11. 1997 /doporučená přístupnost ...,102 min,"4806,",film,"komedie, povídkový","hraný, distribuční",...,None,None,None,None,None,None,None,None,None,None


In [7]:
", ".join(df.columns.to_list())

'Film, Země původu, Copyright, Rok výroby, Premiéra, Minutáž, Režie, Kategorie, Žánr, Typologie, Originální název, Český název, Anglický název, Souběžný název, Pracovní název, Hrají, Postavy, Pomocná režie, Asistent režie, Původní filmový námět, Scénář, Dramaturg, Kamera, Asistent kamery, Architekt, Návrhy kostýmů, Masky, Střih, Zvuk, Producent, Vedoucí výroby, Zástupce vedoucího výroby, Odborný poradce, Hudba, Choreografie, Lokace, Distribuční slogan, Výrobce, S podporou, Nositelé copyrightu, Distribuce, Délka, Distribuční nosič, Poměr stran, Barva, Zvukový, Zvukový systém/formát, Verze, Mluveno, Podtitulky, Úvodní/závěrečné titulky, Anotace, Obsah, Technický scénář, Výtvarník, Animace, Triky, Postprodukce, Návrhy titulků, Pedagogické vedení, Použitá hudba, Studentský film, Původní metráž, Mezititulky, Ocenění, Poznámka, Mluví, Vizuální efekty, Vedoucí produkce, Předloha, Kaskadéři, Text písně, Zpívá, pásmo, Části, Produkce, Výtvarná spolupráce, Nahrál, Partner, Animační technika, Výp

## Pryč s duplikáty

In [8]:
df = df.drop_duplicates(subset="Film")

## Převedení položek oddělených čárkami na seznamy pro snadné .explode() v budoucnu

In [9]:
seznamove_sloupce = [
    "Animace",
    "Animační technika",
    "Architekt",
    "Asistent kamery",
    "Asistent režie",
    "Ateliéry",
    "Autor komentáře",
    "Barva",
    "Distribuční nosič",
    "Dramaturg",
    "Druhá kamera",
    "Hudba",
    "Choreografie",
    "Kamera",
    "Kaskadéři",
    "Koordinátor kaskadérů",
    "Koproducent",
    "Masky",
    "Mezititulky",
    "Mluveno",
    "Natáčecí formát",
    "Návrhy kostýmů",
    "Návrhy titulků",
    "Odborný poradce",
    "Pedagogické vedení",
    "Podtitulky",
    "Pomocná režie",
    "Postprodukce",
    "Producent",
    "Produkce",
    "Předloha",
    "Původní filmový námět",
    "Režie",
    "Scénář",
    "Spolupráce",
    "Střih",
    "Technický scénář",
    "Typologie",
    "Úvodní/závěrečné titulky",
    "Vedoucí produkce",
    "Vedoucí výroby",
    "Verze",
    "Výprava",
    "Výkonná produkce",
    "Výtvarná spolupráce",
    "Výtvarník",
    "Zástupce vedoucího produkce",
    "Zástupce vedoucího výroby",
    "Země původu",
    "Zpívá",
    "Zvuk",
    "Zvukový systém/formát",
    "Žánr"]

In [10]:
seznamove_sloupce_firmy = [
    "Distribuce",
    "Nositelé copyrightu",
    "Partner",
    "S podporou",
    "Triky",
    "Vizuální efekty",
    "Výrobce",
    "Zvláštní efekty"
]

In [11]:
jiz_seznam = [
    "Dirigent",
    "Hudba k písni",
    "Nahrál",
    "Písně",
    "Použitá hudba",
    "Tančí",
    "Text písně",
    "Hudební dramaturgie",
    "Hudební režie",
    "Výběr hudby",
    "Zpěv role"
]

## Funkce pro čištění

In [12]:
def nahrad(bunka, original, nahrada):
    
    if isinstance(bunka, list):
        opravena_bunka = []
        try:
            for b in bunka:
                if b == original:
                    opravena_bunka.append(nahrada)
                elif original in b:
                    opravena_bunka.append(b.replace(original,nahrada))
                else:
                    opravena_bunka.append(b)
        except:
            opravena_bunka = bunka
            
    elif isinstance(bunka, str):
        opravena_bunka = ""
        if bunka == original:
            opravena_bunka = nahrada
        elif original in bunka:
            opravena_bunka = bunka.replace(original, nahrada)
        else:
            opravena_bunka = bunka
    
    else:
        opravena_bunka = bunka
    
    return opravena_bunka

In [13]:
def citlivy_orez(x):
    if isinstance(x, str):
        x = x.strip()
    if isinstance(x, list):
        try:
            x = [y.strip() for y in x]
        except:
            pass
    return(x)

In [14]:
def do_seznamu(polozka):
    try:
        if "," in polozka:
            polozka = str(polozka).split(",")
            polozka = [p.strip() for p in polozka]
    except:
        pass
    return(polozka)

In [15]:
def firmy_do_seznamu(polozka):
    if polozka:
        polozka = re.sub("\s"," ",polozka)
        polozka = re.sub("\n"," ",polozka)
        polozka = polozka.replace(" spol. s "," s. ").replace(" spol.s "," s. ").replace("s.r.o.","s. r. o.").replace("s. r.o.","s. r. o.").replace("a.s.","a. s.").replace(", s. r. o."," s. r. o.").replace(",s. r. o."," s. r. o.").replace(", a. s."," a. s.").replace(",a. s."," a. s.").replace(", v. o. s."," v. o. s.").replace("ČESKÁ TELEVIZE","Česká televize").replace("BONTONFILM","Bontonfilm").replace("FALCON","Falcon")
        polozka = do_seznamu(polozka)
    else:
        polozka = polozka
    return(polozka)

## Čištěníčko

In [16]:
df[df["Režie"].str.contains("české verze Agnieszka Holland /ž/",na=False)]

,Film,Země původu,Copyright,Rok výroby,Premiéra,Minutáž,Režie,Kategorie,Žánr,Typologie,...,Výtvarník pozadí,Výrobní skupina,Zákazník,Vývozní společnost,Umělecké vedení,Zpěv role,Překlad české verze,Rychlost,Přístupnost,cyklus


In [17]:
df["Režie"] = df["Režie"].apply(lambda x: nahrad(x,"české verze", ""))

C:\Users\micha\AppData\Local\Temp\ipykernel_3640\1158444040.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Režie"] = df["Režie"].apply(lambda x: nahrad(x,"české verze", ""))


In [18]:
df[df["Režie"].str.contains("české verze Agnieszka Holland /ž/",na=False)]

,Film,Země původu,Copyright,Rok výroby,Premiéra,Minutáž,Režie,Kategorie,Žánr,Typologie,...,Výtvarník pozadí,Výrobní skupina,Zákazník,Vývozní společnost,Umělecké vedení,Zpěv role,Překlad české verze,Rychlost,Přístupnost,cyklus


In [19]:
df[df["Režie"].str.contains("Agnieszka Holland",na=False)]

,Film,Země původu,Copyright,Rok výroby,Premiéra,Minutáž,Režie,Kategorie,Žánr,Typologie,...,Výtvarník pozadí,Výrobní skupina,Zákazník,Vývozní společnost,Umělecké vedení,Zpěv role,Překlad české verze,Rychlost,Přístupnost,cyklus


In [20]:
df = df.apply(lambda x: citlivy_orez(x))

In [21]:
for s in seznamove_sloupce:
    df[s] = df[s].apply(lambda x: do_seznamu(x))
for s in seznamove_sloupce_firmy:
    df[s] = df[s].apply(lambda x: firmy_do_seznamu(x))

In [22]:
def seznam_lokaci(lokace_hrube):
    lokace_ciste = []
    if lokace_hrube:
        lokace_hrube = lokace_hrube.split("),")
        for x in lokace_hrube:
            if "(" in x:
                x = x + ")"
            x = x.strip()
            x = x.replace("))",")")
            if x[0:2] == ", ":
                x = x[2:]
            if x != ")":
                lokace_ciste.append(x)
    return(lokace_ciste)

In [23]:
df["Lokace"] = df["Lokace"].apply(lambda x: seznam_lokaci(x))

In [24]:
df["Minutáž"] = df["Minutáž"].apply(lambda x: str(x).replace("min","").strip())
df["Minutáž"] = pd.to_numeric(df["Minutáž"], errors="coerce")

In [25]:
def ratio(x):
    if x:
        x = x.replace("1:","")
        x = x.split(", ")
        x = [y.strip() for y in x]
        x = [y.replace(",",".") for y in x]
        x = [pd.to_numeric(y) for y in x]
        if len(x) == 1:
            x = x[0]
    return(x)

In [26]:
df["Poměr stran"] = df["Poměr stran"].apply(lambda x: ratio(x))

In [27]:
def metraz(x):
    if x:
        x = int(x.replace("metrů","").replace(" ",""))
    else:
        x = x
    return x

In [28]:
df["Původní metráž"] = df["Původní metráž"].apply(lambda x: metraz(x))

In [29]:
def kill_whitespace(x):
    def seriously(y):
        y = re.sub("\s+", " ", y)
        return(y)
    if isinstance(x, list):
        x = [seriously(y) for y in x]
    if isinstance(x, str):
        x = seriously(x)
    return(x)

In [30]:
df["Ocenění"] = df["Ocenění"].apply(lambda x: kill_whitespace(x))

In [31]:
def roky_vyroby(x):
    if x:
        zacatek = int(x[:4])
        konec = int(x[-4:])
    else:
        zacatek = None
        konec = None
    return [zacatek, konec]

In [32]:
df["Začátek výroby"] = df["Rok výroby"].apply(lambda x: roky_vyroby(x)[0])
df["Konec výroby"] = df["Rok výroby"].apply(lambda x: roky_vyroby(x)[1])

In [33]:
df = df.sort_values(by = "Copyright")

## Oprava minutáže

In [34]:
df[df["Minutáž"] > 130].groupby(["Film","Copyright"])["Minutáž"].max().nlargest(60)

Film                                                                                                                                              Copyright
Místo v houfu (396595)                                                                                                                            1964         1112.0
Hrdina divokého západu (402185)                                                                                                                   1920          700.0
Zapomenuté transporty (79193)                                                                                                                     2007          341.0
Komunismus a síť, aneb Konec zastupitelské demokracie (402026)                                                                                    2019          337.0
Bohemia docta aneb Labyrint světa a lusthauz srdce /Božská komedie/ (17002)                                                                       2000          254.0
Kdo bude hlída

- Místo v houfu (396595) – Zjevná chyba, nahradíme údajem z ČSDF.
- Venoušek a Stázička, Vojta Lavička: Nahoru a dolů (103360) – dtto
- Hrdina divokého západu (402185) – Složitější: "Humorné příhody Eddieho Pola a Marie Walcampové v „15 epizodách a 142 dílech“. Nejméně špatné ze špatných řešení: nahradíme mediánem za rok 1920, takže to nerozdrbe statistiky.
- Zapomenuté transporty (79193) – vyhazujeme, nechávám jednotlivé díly.

In [35]:
df.loc[df["Film"] == "Místo v houfu (396595)","Minutáž"]=115
df.loc[df["Film"] == "Venoušek a Stázička (395308)","Minutáž"]=68
df.loc[df["Film"] == "Vojta Lavička: Nahoru a dolů (103360)","Minutáž"]=89
df.loc[df["Film"] == "Hrdina divokého západu (402185)","Minutáž"]=df[df["Copyright"] == 1920]["Minutáž"].median().astype(int)
df = df[~(df["Film"] == "Zapomenuté transporty (79193)")]

In [36]:
df = df.drop(labels="cyklus", axis=1)

## Odstranění exilových filmů M. Formana a bondovky

In [37]:
zahranicni = ["Humorista (401985)","Útěk (401824)","Amundsen (401963)",]

In [38]:
df = df[~df.Film.isin(opravy["exil"])]
df = df[~df.Film.isin(zahranicni)]
df = df[df["Země původu"].str.contains("Česk|Rakousko-Uhersko",na=True)]

## Opravy jmen

In [39]:
def replace_names(x):
    if isinstance(x, str):
        if x.strip() in ids_jmena:
            return ids_jmena[x].strip()
        else:
            return x
    elif isinstance(x, list):
        return [replace_names(xi).strip() for xi in x]
    else:
        return x

In [40]:
ids_jmena = opravy["jmena"]
ids_jmena

{'Otakar Vávra': '9507',
 'J. A. Holman': '37290',
 'Hrušínský Rudolf ml.': '2988',
 'Rudolf Hrušínský': '2988'}

In [41]:
for p in profese:
    df[p] = df[p].apply(replace_names)

## Nahrazení ids jmény

In [42]:
persony = pd.read_json(os.path.join("data", "persony.json"))

In [43]:
persony = persony[["Id","Jméno"]]

In [44]:
persony.sample(3)

,Id,Jméno
31117,84191,Karel Fridrich (84191)
28528,7933,Josef Staněk (7933)
11884,18563,Bohemia Big Band (18563)


In [45]:
persony[persony["Id"] == "person"]

,Id,Jméno
35812,person,Čechomor (person)
35813,person,Kevin Mc Kidd (person)
35814,person,Rostislav Novák st. (person)


Tohle nás asi nemusí trápit.

In [46]:
ids_jmena = persony.to_dict(orient="tight")

In [47]:
ids_jmena = {}
for index, row in persony.iterrows():
    ids_jmena[str(row["Id"])] = str(row["Jméno"])

In [48]:
def prejmenuj(bunka):
    
    global ids_jmena
    
    for key, val in ids_jmena.items():

        if bunka != None:
        
            if isinstance(bunka, list):
                opravena_bunka = []
                try:
                    for b in bunka:
                        print(b)
                        if b == str(key):
                            opravena_bunka.append(val)
                        else:
                            opravena_bunka.append(b)
                except:
                    opravena_bunka = bunka

            elif isinstance(bunka, str):
                print(bunka)
                opravena_bunka = ""
                if bunka == str(key):
                    opravena_bunka = val
                else:
                    opravena_bunka = bunka
                    
            else:
                opravena_bunka = bunka

        else:
            opravena_bunka = bunka

    return opravena_bunka

In [49]:
for p in profese:
    df[p] = df[p].apply(replace_names)

## Mazání "/ž/"

In [50]:
df[df["Režie"].str.contains("/ž/",na=False)]

,Film,Země původu,Copyright,Rok výroby,Premiéra,Minutáž,Režie,Kategorie,Žánr,Typologie,...,Výrobní skupina,Zákazník,Vývozní společnost,Umělecké vedení,Zpěv role,Překlad české verze,Rychlost,Přístupnost,Začátek výroby,Konec výroby


In [51]:
def odzen(bunka):
    
    def odz(x):
        return x.replace(" /ž/","").strip()
    
    if isinstance(bunka, list):
        try:
            cista_bunka = [odz(x) for x in bunka]
        except:
            cista_bunka = bunka
            
    if isinstance(bunka, str):
        cista_bunka = odz(bunka)
        
    else:
        cista_bunka = bunka
        
    return cista_bunka

In [52]:
for p in profese:
    df[p] = df[p].apply(lambda x: nahrad(x, " /ž/", ""))

In [53]:
df[df["Režie"].str.contains("/ž/",na=False)]

,Film,Země původu,Copyright,Rok výroby,Premiéra,Minutáž,Režie,Kategorie,Žánr,Typologie,...,Výrobní skupina,Zákazník,Vývozní společnost,Umělecké vedení,Zpěv role,Překlad české verze,Rychlost,Přístupnost,Začátek výroby,Konec výroby


In [54]:
df[df["Režie"].str.contains("Smoczyńska",na=False)]

,Film,Země původu,Copyright,Rok výroby,Premiéra,Minutáž,Režie,Kategorie,Žánr,Typologie,...,Výrobní skupina,Zákazník,Vývozní společnost,Umělecké vedení,Zpěv role,Překlad české verze,Rychlost,Přístupnost,Začátek výroby,Konec výroby


## Odstranění druhých závorek

In [55]:
df["Vedoucí produkce"].drop_duplicates().to_list()

[None,
 ['Václav Eisenhamer (3534)', ''],
 ['Jiří Ježek (4473)', ''],
 ['Pavel Šolc (5420)', ''],
 ['Jaromír Lukáš (27426)', ''],
 ['Pavel Nový (4450)', ''],
 ['Lucie Hertlová (74907)', 'Viktor Mayer (3334)', ''],
 ['Daniela Stašková (34813)',
  'Zdenka Deitchová (1143)',
  'Zdenka Deitchová (kreslená část)'],
 ['Daniela Stašková (34813)', ''],
 ['Šárka Cimbalová (85990)', 'Šárka Cimbalová (vedoucí natáčení)'],
 ['', 'Jana Novosadová (vedoucí natáčení)'],
 ['', 'Jaroslav Kučera (vedoucí natáčení)'],
 ['Ivan Filus (9671)', ''],
 ['Luboš Tůma (9232)', 'Luboš Tůma (vedoucí atelieru)'],
 ['Ivana Jaroschy (3246)', ''],
 ['Antonín Suchánek (12599)', 'Antonín Suchánek (vedoucí natáčení)'],
 ['Jindřich Bareš (7470)', 'Jindřich Bareš (ČT)'],
 ['Markéta Hájková (2388)',
  'Markéta Hájková (vedoucí natáčení)',
  'Andrea Ashton (vedoucí natáčení)'],
 ['Ingrid Klímová (147)', ''],
 ['Michaela Jarošová (1349)', ''],
 ['Aleš Novák (644)', 'Ivan Nagy (34483)', 'Ľubomír Matušek'],
 ['Ľuboslav Žiško (28

In [56]:
def odzavorkuj(bunka):
    
    def odzav(x):
        return re.sub("\(.*[^\d]\)","", x)
    
    if isinstance(bunka, list):
        cista_bunka = [odzav(x).strip() for x in bunka if x]
            
    elif isinstance(bunka, str):
        cista_bunka = odzav(bunka).strip()
        
    else:
        cista_bunka = bunka
        
    return cista_bunka

In [57]:
for p in profese:
    df[p] = df[p].apply(odzavorkuj)

In [58]:
df["Vedoucí produkce"].drop_duplicates().to_list()

[None,
 ['Václav Eisenhamer (3534)'],
 ['Jiří Ježek (4473)'],
 ['Pavel Šolc (5420)'],
 ['Jaromír Lukáš (27426)'],
 ['Pavel Nový (4450)'],
 ['Lucie Hertlová (74907)', 'Viktor Mayer (3334)'],
 ['Daniela Stašková (34813)', 'Zdenka Deitchová (1143)', 'Zdenka Deitchová'],
 ['Daniela Stašková (34813)'],
 ['Šárka Cimbalová (85990)', 'Šárka Cimbalová'],
 ['Jana Novosadová'],
 ['Jaroslav Kučera'],
 ['Ivan Filus (9671)'],
 ['Luboš Tůma (9232)', 'Luboš Tůma'],
 ['Ivana Jaroschy (3246)'],
 ['Antonín Suchánek (12599)', 'Antonín Suchánek'],
 ['Jindřich Bareš (7470)', 'Jindřich Bareš'],
 ['Markéta Hájková (2388)', 'Markéta Hájková', 'Andrea Ashton'],
 ['Ingrid Klímová (147)'],
 ['Michaela Jarošová (1349)'],
 ['Aleš Novák (644)', 'Ivan Nagy (34483)', 'Ľubomír Matušek'],
 ['Ľuboslav Žiško (2808)'],
 ['Pavla Tomsová (7093)'],
 ['Zdenka Fialová (6996)', 'Vladimír Kořínek (2106)'],
 ['Josef Nepovím (7697)'],
 ['Martin Kořínek (1768)', 'Jo Farr (1769)'],
 ['Jiřina Bílá (6711)'],
 ['Adam Polák (2507)'],
 ['

## Kontrola, export

In [59]:
df

,Film,Země původu,Copyright,Rok výroby,Premiéra,Minutáž,Režie,Kategorie,Žánr,Typologie,...,Výrobní skupina,Zákazník,Vývozní společnost,Umělecké vedení,Zpěv role,Překlad české verze,Rychlost,Přístupnost,Začátek výroby,Konec výroby
3763,Polední výstřel na Mariánských hradbách (401709),Rakousko-Uhersko,1898,1898,premiéra 19. 6. 1898 (Výstava architektury a ...,1.0,[Jan Kříženecký (127419)],film,reportáž,"[dokumentární, distribuční]",...,None,None,None,None,None,None,None,None,1898,1898
3758,Cvičení s kužely Sokolů malostranských (401704),Rakousko-Uhersko,1898,1898,premiéra 24. 7. 1898 (Výstava architektury a ...,1.0,[Jan Kříženecký (127419)],film,"[reportáž, sportovní]","[dokumentární, distribuční]",...,None,None,None,None,None,None,None,None,1898,1898
3759,Cyklisté (401705),Rakousko-Uhersko,1898,1898,premiéra léto 1898 (Výstava architektury a in...,1.0,[Jan Kříženecký (127419)],film,reportáž,"[dokumentární, distribuční]",...,None,None,None,None,None,None,None,None,1898,1898
3760,Defilování vojska o Božím těle na Královských ...,Rakousko-Uhersko,1898,1898,premiéra 19. 6. 1898 (Výstava architektury a ...,1.0,[Jan Kříženecký (127419)],film,reportáž,"[dokumentární, distribuční]",...,None,None,None,None,None,None,None,None,1898,1898
3761,Hanácké banderium (401707),Rakousko-Uhersko,1898,1898,premiéra 24. 7. 1898 (Výstava architektury a ...,1.0,[Jan Kříženecký (127419)],film,reportáž,"[dokumentární, distribuční]",...,None,None,None,None,None,None,None,None,1898,1898
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4211,Mimořádná událost (402176),Česká republika,2022,2021,"slavnostní premiéra 2. 2. 2022 (kino Lucerna,...",102.0,[Jiří Havelka (28977)],film,"[komedie, satira, bláznivá komedie]","[hraný, distribuční]",...,None,None,None,None,None,None,None,None,2021,2021
4208,Po čem muži touží 2 (402173),"[Česká republika, Slovensko]",2022,2021,slavnostní premiéra 20. 4. 2022 (multiplex Ci...,96.0,[Rudolf Havlík (79764)],film,"[fantastický, komedie]","[hraný, distribuční]",...,None,None,None,None,None,None,None,None,2021,2021
4222,Až zařve lev (402204),Česká republika,2022,2020—2021,"slavnostní premiéra 5. 4. 2022 (kino Lucerna,...",104.0,[Jan Svatoš (77843)],film,"[historický, životopisný, esej]","[dokumentární, distribuční]",...,None,None,None,None,None,None,None,None,2020,2021
4232,Planeta Praha (402218),"[Česká republika, Slovensko]",2022,2020—2021,"slavnostní premiéra 2. 8. 2022 (kino Aero, Pr...",89.0,[Jan Hošek (5629)],film,"[přírodopisný, rodinný]","[dokumentární, distribuční]",...,None,None,None,None,None,None,None,None,2020,2021


In [60]:
with open(os.path.join("seznamove_sloupce.txt"), "w+", encoding = "utf-8") as s_sloupce:
    s_sloupce.writelines("\n".join(seznamove_sloupce))
    s_sloupce.writelines("\n")
    s_sloupce.writelines("\n".join(seznamove_sloupce_firmy))
    s_sloupce.writelines("\n")
    s_sloupce.writelines("\n".join(jiz_seznam))

In [61]:
with open(os.path.join("dlouhometrazni.txt"), "w+", encoding = "utf-8") as dlouhometrazni:
    dlouhometrazni.writelines("\n".join(df[df["Délka"] == "dlouhometrážní"]["Film"].drop_duplicates().to_list()))

In [62]:
df.shape

(4508, 132)

In [63]:
df

,Film,Země původu,Copyright,Rok výroby,Premiéra,Minutáž,Režie,Kategorie,Žánr,Typologie,...,Výrobní skupina,Zákazník,Vývozní společnost,Umělecké vedení,Zpěv role,Překlad české verze,Rychlost,Přístupnost,Začátek výroby,Konec výroby
3763,Polední výstřel na Mariánských hradbách (401709),Rakousko-Uhersko,1898,1898,premiéra 19. 6. 1898 (Výstava architektury a ...,1.0,[Jan Kříženecký (127419)],film,reportáž,"[dokumentární, distribuční]",...,None,None,None,None,None,None,None,None,1898,1898
3758,Cvičení s kužely Sokolů malostranských (401704),Rakousko-Uhersko,1898,1898,premiéra 24. 7. 1898 (Výstava architektury a ...,1.0,[Jan Kříženecký (127419)],film,"[reportáž, sportovní]","[dokumentární, distribuční]",...,None,None,None,None,None,None,None,None,1898,1898
3759,Cyklisté (401705),Rakousko-Uhersko,1898,1898,premiéra léto 1898 (Výstava architektury a in...,1.0,[Jan Kříženecký (127419)],film,reportáž,"[dokumentární, distribuční]",...,None,None,None,None,None,None,None,None,1898,1898
3760,Defilování vojska o Božím těle na Královských ...,Rakousko-Uhersko,1898,1898,premiéra 19. 6. 1898 (Výstava architektury a ...,1.0,[Jan Kříženecký (127419)],film,reportáž,"[dokumentární, distribuční]",...,None,None,None,None,None,None,None,None,1898,1898
3761,Hanácké banderium (401707),Rakousko-Uhersko,1898,1898,premiéra 24. 7. 1898 (Výstava architektury a ...,1.0,[Jan Kříženecký (127419)],film,reportáž,"[dokumentární, distribuční]",...,None,None,None,None,None,None,None,None,1898,1898
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4211,Mimořádná událost (402176),Česká republika,2022,2021,"slavnostní premiéra 2. 2. 2022 (kino Lucerna,...",102.0,[Jiří Havelka (28977)],film,"[komedie, satira, bláznivá komedie]","[hraný, distribuční]",...,None,None,None,None,None,None,None,None,2021,2021
4208,Po čem muži touží 2 (402173),"[Česká republika, Slovensko]",2022,2021,slavnostní premiéra 20. 4. 2022 (multiplex Ci...,96.0,[Rudolf Havlík (79764)],film,"[fantastický, komedie]","[hraný, distribuční]",...,None,None,None,None,None,None,None,None,2021,2021
4222,Až zařve lev (402204),Česká republika,2022,2020—2021,"slavnostní premiéra 5. 4. 2022 (kino Lucerna,...",104.0,[Jan Svatoš (77843)],film,"[historický, životopisný, esej]","[dokumentární, distribuční]",...,None,None,None,None,None,None,None,None,2020,2021
4232,Planeta Praha (402218),"[Česká republika, Slovensko]",2022,2020—2021,"slavnostní premiéra 2. 8. 2022 (kino Aero, Pr...",89.0,[Jan Hošek (5629)],film,"[přírodopisný, rodinný]","[dokumentární, distribuční]",...,None,None,None,None,None,None,None,None,2020,2021


In [64]:
df.to_json(os.path.join("data", "filmy.json"), orient="records")

In [65]:
import zipfile
try:
    import zlib
    mode= zipfile.ZIP_DEFLATED
except:
    mode= zipfile.ZIP_STORED